In [ ]:
!pip install --upgrade langchain langgraph langchain-openai pydot graphviz
import os, getpass, warnings; warnings.filterwarnings("ignore")

#PromptTemplate

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("Say hello to {name} in Korean. Answer only in Korean.")
print("템플릿 변수:", prompt.input_variables)
print(prompt.format(name="Alice"))


#LLM

In [ ]:
!pip install langchain_huggingface

In [ ]:
!huggingface-cli login

In [ ]:
from langchain_huggingface import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-base",
    task="text2text-generation",
    pipeline_kwargs={
        "max_new_tokens": 256,
        "temperature": 0.7,
    },
    device_map="auto",  # GPU가 없으면 자동으로 CPU 사용
)

# 사용법은 동일합니다.
print("LangChain 파이프라인으로 모델을 호출합니다...")
response = llm.invoke("LangChain에 대해 한 문장으로 설명해줘.")
print(response)

#Chain

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers.string import StrOutputParser

# 1) LLM 설정
llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-base",
    task="text2text-generation",
    pipeline_kwargs={
        "max_new_tokens": 256,
        "temperature": 0.7,
    },
    device_map="auto",  # GPU가 없으면 자동으로 CPU 사용
)

# 2) 감정 분석용 프롬프트 템플릿
prompt = PromptTemplate.from_template(
    "다음 문장의 감정(sentiment)을 분석하고, -1.0(매우 부정)부터 +1.0(매우 긍정) 사이의 부동소수점 점수와 간단한 라벨('positive', 'neutral', 'negative')을"
    " 다음 형식으로 출력하세요:\n"
    "SCORE: <score>\n"
    "LABEL: <label>\n\n"
    "문장: \"{text}\""
)

# 3) 체인 조합 (템플릿 → LLM)
# 문자열 파서가 없으면 생성 응답 외의 메타데이터도 다 출력된다.
chain = (
    prompt
    | llm
)

# 4) 실행 예시
result = chain.invoke({
    "text": "오늘 날씨는 참 좋지만, 지하철이 너무 붐벼서 힘들었어요."
})
print(result)

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers.string import StrOutputParser

# 1) LLM 설정
llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-base",
    task="text2text-generation",
    pipeline_kwargs={
        "max_new_tokens": 256,
        "temperature": 0.7,
    },
    device_map="auto",  # GPU가 없으면 자동으로 CPU 사용
)

# 2) 감정 분석용 프롬프트 템플릿
prompt = PromptTemplate.from_template(
    "다음 문장의 감정(sentiment)을 분석하고, -1.0(매우 부정)부터 +1.0(매우 긍정) 사이의 부동소수점 점수와 간단한 라벨('positive', 'neutral', 'negative')을"
    " 다음 형식으로 출력하세요:\n"
    "SCORE: <score>\n"
    "LABEL: <label>\n\n"
    "문장: \"{text}\""
)

# 3) 체인 조합 (템플릿 → LLM → 문자열 파서)
chain = (
    prompt
    | llm
    | StrOutputParser()
)

# 4) 실행 예시
result = chain.invoke({
    "text": "오늘 날씨는 참 좋지만, 지하철이 너무 붐벼서 힘들었어요."
})
print(result)

#프로젝트

기초 준비

In [ ]:
# 필요한 라이브러리들을 가져옵니다.
import os
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from typing import TypedDict
from langgraph.graph import StateGraph, END

# --- LLM 및 감정 분석 체인 설정 ---

# LLM을 초기화합니다.
llm = HuggingFacePipeline.from_model_id(
    model_id="mrm8488/t5-base-finetuned-emotion",
    task="text2text-generation",
    pipeline_kwargs={
        "max_new_tokens": 10,
        "temperature": 0.5,
    },
    device_map="auto"
)


# LLM에게 어떤 작업을 시킬지 프롬프트로 정의합니다.
prompt = PromptTemplate.from_template(
    "다음 문장의 감정을 분석하세요. 결과는 다음 형식을 따르세요:\n"
    "SCORE: <숫자: -1.0(매우 부정) ~ +1.0(매우 긍정)>\n"
    "LABEL: <positive|neutral|negative>\n\n"
    "예시:\n"
    "문장: \"이 제품은 정말 마음에 들어요.\"\n"
    "SCORE: 0.9\n"
    "LABEL: positive\n\n"
    "문장: \"{text}\""
)


# 프롬프트, LLM, 출력 파서를 파이프라인처럼 연결하여 '체인'을 만듭니다.
sentiment_analysis_chain = prompt | llm | StrOutputParser()

print("✅ 1단계: 기초 준비 완료!")

상태(state)

In [ ]:
# 그래프의 상태(State) 구조를 정의합니다.
# 각 노드를 거치면서 이 상태 객체에 데이터가 채워지고 수정됩니다.
class GraphState(TypedDict):
    user_input: str      # 사용자가 입력한 원본 문장
    sentiment_score: float # LLM이 분석한 감정 점수
    sentiment_label: str   # LLM이 분석한 감정 라벨 ('positive', 'neutral', 'negative')

print("✅ 2단계: 상태 정의 완료!")

노드(node)

In [ ]:
# 1. 감정 분석을 수행하는 메인 노드
def analyze_sentiment_node(state: GraphState) -> GraphState:
    print("\n--- [노드 실행] 감정 분석 시작 ---")
    user_text = state["user_input"]

    # LLM 호출
    result_str = sentiment_analysis_chain.invoke({"text": user_text})
    print(f"LLM 분석 결과:\n{result_str}")

    # 모델이 출력한 감정 라벨 (예: "joy") 에서 긍/부/중립 라벨로 매핑
    label_map = {
        "joy": "positive",
        "love": "positive",
        "surprise": "positive",
        "anger": "negative",
        "sadness": "negative",
        "fear": "negative",
    }

    label_raw = result_str.strip().lower()
    mapped_label = label_map.get(label_raw, "neutral")

    # 상태 업데이트
    state["sentiment_score"] = 0.0  # 점수는 없는 모델이므로 기본값 사용
    state["sentiment_label"] = mapped_label

    return state


# 2. 각 결과에 따라 간단한 메시지를 출력하는 노드들
def positive_node(state: GraphState):
    print("--- [노드 실행] 긍정적인 반응 ---")
    print("😀 긍정적인 내용이네요! 좋은 하루 보내세요.")

def neutral_node(state: GraphState):
    print("--- [노드 실행] 중립적인 반응 ---")
    print("😐 중립적인 내용이군요. 알려주셔서 감사합니다.")

def negative_node(state: GraphState):
    print("--- [노드 실행] 부정적인 반응 ---")
    print("😟 부정적인 경험을 하셨군요. 괜찮으신가요?")

print("✅ 3단계: 노드 정의 완료!")

그래프 구성 및 엣지(Edge) 연결

In [ ]:
# 그래프의 흐름을 결정하는 라우팅(routing) 함수
def route_by_sentiment(state: GraphState) -> str:
    """상태의 sentiment_label을 보고 다음 노드의 이름을 반환합니다."""
    label = state["sentiment_label"]
    print(f"\n--- [분기] '{label}' 라벨에 따라 다음 경로 결정 ---")
    return label

# --- 그래프 설계 시작 ---

# 1. 상태(State) 모델을 기반으로 그래프 객체를 생성합니다.
g = StateGraph(GraphState)

# 2. 그래프에 노드들을 추가합니다. (이름, 실행할 함수)
g.add_node("analyze_sentiment", analyze_sentiment_node)
g.add_node("positive_path", positive_node)
g.add_node("neutral_path", neutral_node)
g.add_node("negative_path", negative_node)

# 3. 진입점(Entry Point)을 설정합니다.
g.set_entry_point("analyze_sentiment")

# 4. 조건부 엣지(Conditional Edge)를 추가합니다.
# 'analyze_sentiment' 노드가 끝난 후, 'route_by_sentiment' 함수의 결과에 따라 다음 노드로 분기합니다.
g.add_conditional_edges(
    "analyze_sentiment", #start_node_name
    route_by_sentiment, #path_decider
    path_map={
        "positive": "positive_path",
        "neutral": "neutral_path",
        "negative": "negative_path"
    }
)

# 5. 각 분기 노드에서 작업이 끝나면 그래프를 종료(END)하도록 엣지를 추가합니다.
g.add_edge("positive_path", END)
g.add_edge("neutral_path", END)
g.add_edge("negative_path", END)

print("✅ 4단계: 그래프 설계 완료!")

그래프 컴파일 및 호출

In [ ]:
# 그래프 컴파일
graph = g.compile()

# 실행할 문장
input_text = "오늘 날씨는 참 좋지만, 지하철이 너무 붐벼서 힘들었어요."

# 그래프 실행
final_state = graph.invoke({"user_input": input_text})

print("\n---")
print("✅ 5단계: 그래프 실행 완료!")
print("최종 상태:", final_state)

In [ ]:
# 필요한 라이브러리들을 가져옵니다.
import os
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from typing import TypedDict
from langgraph.graph import StateGraph, END

# --- LLM 및 감정 분석 체인 설정 ---

# LLM을 초기화합니다.
llm = HuggingFacePipeline.from_model_id(
    model_id="mrm8488/t5-base-finetuned-emotion",
    task="text2text-generation",
    pipeline_kwargs={
        "max_new_tokens": 10,
        "temperature": 0.5,
    },
    device_map="auto"
)

In [ ]:
# LLM에게 어떤 작업을 시킬지 프롬프트로 정의합니다.
prompt = PromptTemplate.from_template(
    "다음 문장의 감정(sentiment)을 분석하고, -1.0(매우 부정)부터 +1.0(매우 긍정) 사이의 점수와 "
    "라벨('positive', 'neutral', 'negative')을 다음 형식으로 출력하세요:\n"
    "SCORE: <score>\n"
    "LABEL: <label>\n\n"
    "문장: \"{text}\""
)


# 프롬프트, LLM, 출력 파서를 파이프라인처럼 연결하여 '체인'을 만듭니다.
sentiment_analysis_chain = prompt | llm | StrOutputParser()

print("✅ 1단계: 기초 준비 완료!")

In [ ]:
# 그래프의 상태(State) 구조를 정의합니다.
# 각 노드를 거치면서 이 상태 객체에 데이터가 채워지고 수정됩니다.
class GraphState(TypedDict):
    user_input: str      # 사용자가 입력한 원본 문장
    sentiment_score: float # LLM이 분석한 감정 점수
    sentiment_label: str   # LLM이 분석한 감정 라벨 ('positive', 'neutral', 'negative')

print("✅ 2단계: 상태 정의 완료!")


# 1. 감정 분석을 수행하는 메인 노드
# def analyze_sentiment_node(state: GraphState) -> GraphState:
#     print("\n--- [노드 실행] 감정 분석 시작 ---")
#     user_text = state["user_input"]

#     # LLM 호출
#     result_str = sentiment_analysis_chain.invoke({"text": user_text})
#     print(f"LLM 분석 결과:\n{result_str}")

#     # 모델이 출력한 감정 라벨 (예: "joy") 에서 긍/부/중립 라벨로 매핑
#     label_map = {
#         "joy": "positive",
#         "love": "positive",
#         "surprise": "positive",
#         "anger": "negative",
#         "sadness": "negative",
#         "fear": "negative",
#     }

#     label_raw = result_str.strip().lower()
#     mapped_label = label_map.get(label_raw, "neutral")

#     # 상태 업데이트
#     state["sentiment_score"] = 0.0  # 점수는 없는 모델이므로 기본값 사용
#     state["sentiment_label"] = mapped_label

#     return state

def analyze_sentiment_node(state: GraphState) -> GraphState:
    print("\n--- [노드 실행] 감정 분석 시작 ---")
    user_text = state["user_input"]

    result_str = sentiment_analysis_chain.invoke({"text": user_text})
    print(f"LLM 분석 결과:\n{result_str}")

    # 점수와 라벨 초기화
    score = 0.0
    label = "neutral"

    try:
        parsed_output = {}
        for line in result_str.strip().split("\n"):
            if ":" in line:
                key, value = line.split(":", 1)
                parsed_output[key.strip().upper()] = value.strip()

        # SCORE와 LABEL이 모두 존재할 때만 사용
        if "SCORE" in parsed_output and "LABEL" in parsed_output:
            score = float(parsed_output["SCORE"])
            label = parsed_output["LABEL"].lower()
    except Exception as e:
        print(f"[파싱 오류] {e}, 기본값 사용")

    state["sentiment_score"] = score
    state["sentiment_label"] = label

    return state


# 2. 각 결과에 따라 간단한 메시지를 출력하는 노드들
def positive_node(state: GraphState):
    print("--- [노드 실행] 긍정적인 반응 ---")
    print("😀 긍정적인 내용이네요! 좋은 하루 보내세요.")

def neutral_node(state: GraphState):
    print("--- [노드 실행] 중립적인 반응 ---")
    print("😐 중립적인 내용이군요. 알려주셔서 감사합니다.")

def negative_node(state: GraphState):
    print("--- [노드 실행] 부정적인 반응 ---")
    print("😟 부정적인 경험을 하셨군요. 괜찮으신가요?")

print("✅ 3단계: 노드 정의 완료!")


# 그래프의 흐름을 결정하는 라우팅(routing) 함수
def route_by_sentiment(state: GraphState) -> str:
    """상태의 sentiment_label을 보고 다음 노드의 이름을 반환합니다."""
    label = state["sentiment_label"]
    print(f"\n--- [분기] '{label}' 라벨에 따라 다음 경로 결정 ---")
    return label

# --- 그래프 설계 시작 ---

# 1. 상태(State) 모델을 기반으로 그래프 객체를 생성합니다.
g = StateGraph(GraphState)

# 2. 그래프에 노드들을 추가합니다. (이름, 실행할 함수)
g.add_node("analyze_sentiment", analyze_sentiment_node)
g.add_node("positive_path", positive_node)
g.add_node("neutral_path", neutral_node)
g.add_node("negative_path", negative_node)

# 3. 진입점(Entry Point)을 설정합니다.
g.set_entry_point("analyze_sentiment")

# 4. 조건부 엣지(Conditional Edge)를 추가합니다.
# 'analyze_sentiment' 노드가 끝난 후, 'route_by_sentiment' 함수의 결과에 따라 다음 노드로 분기합니다.
g.add_conditional_edges(
    "analyze_sentiment", #start_node_name
    route_by_sentiment, #path_decider
    path_map={
        "positive": "positive_path",
        "neutral": "neutral_path",
        "negative": "negative_path"
    }
)

# 5. 각 분기 노드에서 작업이 끝나면 그래프를 종료(END)하도록 엣지를 추가합니다.
g.add_edge("positive_path", END)
g.add_edge("neutral_path", END)
g.add_edge("negative_path", END)

print("✅ 4단계: 그래프 설계 완료!")

In [ ]:
# 그래프 컴파일
graph = g.compile()

# 실행할 문장
input_text = "오늘 날씨는 참 좋지만, 지하철이 너무 붐벼서 힘들었어요."

# 그래프 실행
final_state = graph.invoke({"user_input": input_text})

print("\n---")
print("✅ 5단계: 그래프 실행 완료!")
print("최종 상태:", final_state)

그래프 구조 출력하기(grandalf, mermaid)

In [ ]:
!pip install grandalf

In [ ]:
import grandalf

print(graph.get_graph().draw_ascii())

In [ ]:
# Mermaid 포맷의 코드를 생성합니다.
mermaid_code = graph.get_graph().draw_mermaid()
print(mermaid_code)

#과제

In [ ]:
!pip install -U langchain langchain-community transformers

In [ ]:
from typing import TypedDict
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
from langgraph.graph import StateGraph, END

# 1. 모델 준비 및 래핑
sentiment_pipeline = pipeline(
    "text-classification",
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
    top_k=1
)
llm = HuggingFacePipeline(pipeline=sentiment_pipeline)

# 2. PromptTemplate 사용 (채점 기준 만족)
prompt = PromptTemplate.from_template("Analyze the emotion of the following input:\n{text}")
chain = prompt | llm | StrOutputParser()

# 3. 상태 정의 (채점 기준 만족)
class GraphState(TypedDict):
    user_input: str
    sentiment_label: str
    recommended_news: str

# 4. 노드 정의 (채점 기준 만족)

# 감정 분석 노드
def analyze_sentiment(state: GraphState) -> GraphState:
    print("\n[분석 중] 사용자 입력:", state["user_input"])
    outputs = sentiment_pipeline(state["user_input"])
    print("파이프라인 출력:", outputs)

    # 중첩 리스트 방지
    result = outputs[0][0] if isinstance(outputs[0], list) else outputs[0]
    label = result["label"].lower()
    print("예측 감정:", label)

    # 감정 라벨을 positive/neutral/negative로 매핑
    label_map = {
        "positive": "positive",
        "neutral": "neutral",
        "negative": "negative",
        "joy": "positive",
        "love": "positive",
        "surprise": "positive",
        "anger": "negative",
        "sadness": "negative",
        "fear": "negative"
    }
    mapped_label = label_map.get(label, "neutral")
    state["sentiment_label"] = mapped_label
    return state


# 감정에 따른 뉴스 추천 노드들
def recommend_positive_news(state: GraphState) -> GraphState:
    state["recommended_news"] = "✨ 오늘의 긍정 뉴스: 과학 기술의 발전으로 암 치료법이 새롭게 개발되었습니다!"
    return state

def recommend_neutral_news(state: GraphState) -> GraphState:
    state["recommended_news"] = "📊 오늘의 정보 뉴스: 미국의 기준금리가 0.25%p 인상되었습니다."
    return state

def recommend_negative_news(state: GraphState) -> GraphState:
    state["recommended_news"] = "🌿 힐링 뉴스: 제주 바다의 돌고래 떼가 다시 돌아왔습니다."
    return state

# 5. 조건부 분기 (채점 기준 만족)
def route_sentiment(state: GraphState) -> str:
    return state["sentiment_label"]

# 6. 그래프 구성
graph_builder = StateGraph(GraphState)
graph_builder.set_entry_point("analyze")
graph_builder.add_node("analyze", analyze_sentiment)
graph_builder.add_node("positive_news", recommend_positive_news)
graph_builder.add_node("neutral_news", recommend_neutral_news)
graph_builder.add_node("negative_news", recommend_negative_news)

# 조건부 경로 추가
graph_builder.add_conditional_edges(
    "analyze",
    route_sentiment,
    {
        "positive": "positive_news",
        "neutral": "neutral_news",
        "negative": "negative_news"
    }
)

# 종료 처리
graph_builder.add_edge("positive_news", END)
graph_builder.add_edge("neutral_news", END)
graph_builder.add_edge("negative_news", END)

# 그래프 컴파일
graph = graph_builder.compile()

# 실행 예시
user_text = "오늘은 정말 기분이 좋고 뿌듯한 하루였어요."
result_state = graph.invoke({"user_input": user_text})

print("\n🎯 감정 라벨:", result_state["sentiment_label"])
print("📰 추천 뉴스:", result_state["recommended_news"])


In [ ]:
import grandalf

print(graph.get_graph().draw_ascii())

In [ ]:
mermaid_code = graph.get_graph().draw_mermaid()
print(mermaid_code)